In [1]:
# Import dependencies
import pandas as pd
import requests
import numpy as np

In [2]:
# Read data from csv
filepath = 'Real-Time_Traffic_Incident_Reports_20240202.csv'
df = pd.read_csv(filepath)

C:\Users\Happy\AppData\Local\Temp\ipykernel_30576\3220555041.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [3]:
# View
df.head()

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date,Agency
0,121E4F6B2D93D3F508359C8700406A1B992733AF_16597...,08/05/2022 09:37:14 PM +0000,TRFC HAZD/ DEBRIS,POINT (0 0),0.000000,0.000000,tra,ARCHIVED,08/05/2022 09:50:03 PM +0000,NaN
1,B852035718A45A6479B38C26FA96B28B0C9A8A56_16615...,08/26/2022 09:05:01 PM +0000,TRFC HAZD/ DEBRIS,POINT (0 0),0.000000,0.000000,900 S FM 973,ARCHIVED,08/26/2022 09:35:03 PM +0000,NaN
2,B7CA5DF711D07BA6D03B6EF004402A6594C77CD6_16622...,09/04/2022 04:16:02 AM +0000,COLLISION,POINT (0 0),0.000000,0.000000,12009 W US 290 HWY,ARCHIVED,09/04/2022 05:40:03 AM +0000,NaN
3,5F5898E4726001663BA5A126B313B03B1AED3F07_16631...,09/14/2022 03:14:19 PM +0000,Crash Urgent,POINT (-97.711561 30.307396),30.307396,-97.711561,4900 N Ih 35 Nb,ARCHIVED,09/14/2022 03:30:04 PM +0000,NaN
4,3791C82875F2B544CEFA8FBE35109575B1431A30_16631...,09/14/2022 01:50:59 PM +0000,TRFC HAZD/ DEBRIS,POINT (-97.820007 30.233228),30.233228,-97.820007,4953-4973 W Us 290 Hwy Eb,ARCHIVED,09/14/2022 02:10:02 PM +0000,NaN


In [4]:
# Drop new Agency column since most values are NaN
df = df.drop(columns='Agency')

In [5]:
# Drop rows wth NaN values
df.dropna(axis=0, inplace=True)

In [6]:
# Remove Latitude data out of range
df = df.loc[df['Latitude'].abs() < 90]

In [7]:
# Convert dates to datetime format
df['Published Date'] = pd.to_datetime(df['Published Date'].str[:22], format='%m/%d/%Y %I:%M:%S %p')
df['Status Date'] = pd.to_datetime(df['Status Date'].str[:22], format='%m/%d/%Y %I:%M:%S %p')

In [8]:
pd.DataFrame(df['Issue Reported'].value_counts())

,count
Issue Reported,
Traffic Hazard,107513
Crash Urgent,85713
Crash Service,49908
COLLISION,35048
TRFC HAZD/ DEBRIS,26287
LOOSE LIVESTOCK,9897
COLLISION WITH INJURY,9010
zSTALLED VEHICLE,8745
Stalled Vehicle,6912


In [9]:
# Combine Issue Reported values based on similar categories

# Combine Fleet Vehicle categories
df.loc[df['Issue Reported'].isin(['FLEET ACC/ FATAL','FLEET ACC/ INJURY']),'Issue Reported'] = 'Fleet Vehicle Accident'

# Combine Traffic Impediments
df.loc[df['Issue Reported'].isin(['BLOCKED DRIV/ HWY','OBSTRUCT HWY']),'Issue Reported'] = 'Traffic Impediment'

# Combine Traffic Hazards
df.loc[df['Issue Reported'].isin(['TRFC HAZD/ DEBRIS','N / HZRD TRFC VIOL']),'Issue Reported'] = 'Traffic Hazard'

In [10]:
# Drop unknown values with few entries
df.drop(df.loc[df['Issue Reported'].isin(['COLLISN / FTSRA','AUTO/ PED'])].index, inplace=True)

# Rename value/fix typo for clarity
df.loc[df['Issue Reported'] == 'COLLISN/ LVNG SCN','Issue Reported'] = 'Collision/Leaving Scene'
df.loc[df['Issue Reported'] == 'zSTALLED VEHICLE','Issue Reported'] = 'Stalled Vehicle'

# Make capitalization uniform
df['Issue Reported'] = df['Issue Reported'].map(str.title)

In [11]:
pd.DataFrame(df['Issue Reported'].value_counts())

,count
Issue Reported,
Traffic Hazard,133811
Crash Urgent,85713
Crash Service,49908
Collision,35048
Stalled Vehicle,15657
Loose Livestock,9897
Collision With Injury,9010
Traffic Impediment,6892
Collision/Leaving Scene,6683


In [12]:
pd.DataFrame(df['Address'].value_counts()).head(50)

,count
Address,
E Parmer Ln & Dessau Rd,228
9300 S Ih 35 Svrd Sb,217
W Slaughter Ln & Menchaca Rd,200
710 E Ben White Blvd Svrd Wb,199
E Riverside Dr & S Pleasant Valley Rd,189
Fm 969 Rd & N Fm 973 Rd,187
Dessau Rd & E Parmer Ln,178
5500-6631 N Mopac Expy Nb,167
W Braker Ln & N Lamar Blvd,164


In [13]:
# Export all cleaned data
df.to_csv('all_time_traffic_incidents_20240119.csv',index=False)

In [14]:
# Filter out dates to only 2023
df_sample = df.loc[(df['Published Date'] > pd.Timestamp('12/31/2022')) & (df['Published Date'] < pd.Timestamp('01/01/2024'))]

# Export 2023 data
df_sample.to_csv('austin_traffic_incident_reports_2023_sample.csv')